In [1]:
import torch
from model.data_loader import (
    read_images
)
from training.train import train_model
import numpy as np
from torch.utils.data import DataLoader, Dataset

/users/local/c24cavig/MLforEarthObservation/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path_index_h = "data/NDBI_20m.img"
path_temperature_c = "data/LST_100m.img"
path_index_c = "data/NDBI_100m.img"

I_H, cols_h, rows_h, crs_h, transform_h = read_images(path_index_h)
print(f"Image NDBI_20m.img - Width: {cols_h}, Height: {rows_h}")

T_C, cols_t, rows_t, crs_t, transform_t = read_images(path_temperature_c)
print(f"Image LST_100m.img - Width: {cols_t}, Height: {rows_t}")

I_C, cols_c, rows_c, crs_c, transform_c = read_images(path_index_c)
print(f"Image NDBI_100m.img - Width: {cols_c}, Height: {rows_c}")

Image NDBI_20m.img - Width: 150, Height: 150
Image LST_100m.img - Width: 30, Height: 30
Image NDBI_100m.img - Width: 30, Height: 30


In [3]:
sIc=np.std(I_C)
sTc=np.std(T_C)
mIc=np.mean(I_C)
mTc=np.mean(T_C)
I_C=(I_C-np.mean(I_C))/np.std(I_C)
T_C=(T_C-np.mean(T_C))/np.std(T_C)

#Create dataloader
signals_all = np.stack((I_C.flatten(), T_C.flatten()),axis=1)
signals_all = signals_all[:,:,np.newaxis]

class TrajDataSet(Dataset):
    def __init__(self,  traj, transform=None):
        self.traj = traj
        self.transform = transform

    def __len__(self):
        return self.traj.shape[0]

    def __getitem__(self, idx):
        # select coordinates
        sample = self.traj[idx,:,:]
        if self.transform:
            sample = self.transform(sample)

        return sample

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""
    def __call__(self, sample):
        if(cuda):
            return torch.FloatTensor(sample).cuda()
        else:
            return torch.FloatTensor(sample)
        #return torch.FloatTensor(sample)

# hyperparameters
batch_size = 128
batches=signals_all.shape[0]/batch_size

## reduce size dataset
train_set = TrajDataSet(signals_all, transform= ToTensor())
train_loader = DataLoader(train_set, batch_size=batch_size, num_workers = 0, shuffle = True, drop_last=False)

In [9]:
model2, input_stats, target_stats = train_model(I_C, T_C, epochs=50, batch_size=32)

torch.save(model2.state_dict(), "training/swin_transformer_model.pth")

HELLO


RuntimeError: shape '[1, 5, 5, -1]' is invalid for input of size 150528

In [4]:
from model.swin_transformer import SwinTransformerV1
from model.data_loader import prepare_dataloader

device = "cuda"

In [5]:
train_loader, input_stats, target_stats = prepare_dataloader(I_C, T_C, 32)

# Cria o modelo simplificado
model = SwinTransformerV1(
    img_size=224,
    in_chans=1,
    out_chans=1
).to(device)

HELLO


In [7]:
for batch in train_loader:
    inputs = batch['input'].to(device)
    targets = batch['target'].to(device)
    break

In [8]:
x = inputs

if x.dim() == 3:
    x = x.unsqueeze(1)
    
# Extrai features com Swin Transformer
features = model.swin.forward_features(x)

# Redimensiona as features para formato de imagem
batch_size = x.shape[0]
h = w = int(features.shape[1] ** 0.5)  # Assume features são quadradas
features = features.view(batch_size, h, w, -1).permute(0, 4, 1, 2)

# Aplica convolução final
output = model.final_conv(features)

RuntimeError: shape '[1, 5, 5, -1]' is invalid for input of size 150528

In [13]:
features.shape

torch.Size([1, 28, 28, 192])